The purpose of this notebook is to include additional attributes to the `01_metadata.csv` to facilitate search and discovery. The types of attributes may include huc identifiers, nhd stream comids, etc. 

In [1]:
import pandas
from pathlib import Path
import pyarrow.parquet as pq
import requests
#from tqdm import tqdm
from tqdm.notebook import tqdm

In [2]:
table = pq.read_table(
    "data_parquet/metadata.parquet",
    use_pandas_metadata=False
)

df = table.to_pandas()

In [3]:
df.dtypes

STREAM_ID             object
SourceID              object
site name             object
source                object
latitude_wgs84       float64
longitude_wgs84      float64
State Code            object
State                 object
drainagearea_sqkm    float64
geometry              object
dtype: object

Loop over each site in the metadata file and query HUC attributes from geoconnex

In [85]:
def call_geoconnex(url):
    req = requests.get(url)
    
    # if a non-200 code is returned, skip the site.
    if req.status_code != 200:
        print(f'Error processing gauge {row.STREAM_ID}: {url}')
        errors[row.STREAM_ID] = req.text

    # get the response in JSON to make processing the data easier
    return req.json()

def get_geoconnex_metadata(df, 
                           service='hu06', 
                           result_key='huc6', 
                           buffer = 0.00001,
                           buffer_increment=0.0005,
                           retries = 5,
                           base_url='https://reference.geoconnex.us/collections', 
                           output_format='json'):
    attributes = {}
    errors = {}
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
 
        lat = row.latitude_wgs84
        lon = row.longitude_wgs84
        
        # build a bounding box that surrounds the lat/lon using the format
        # min_lon, min_lat, max_lon, max_lat. Buffer these points.
        bbox = f'{lon-buffer},{lat-buffer},{lon+buffer},{lat+buffer}'
    
        url = f'{base_url}/{service}/items?f={output_format}&bbox={bbox}' 

        res = call_geoconnex(url)
        # req = requests.get(url)
    
        # # if a non-200 code is returned, skip the site.
        # if req.status_code != 200:
        #     print(f'Error processing gauge {row.STREAM_ID}: {url}')
        #     errors[row.STREAM_ID] = req.text
        #     continue
    
        # # get the response in JSON to make processing the data easier
        # res = req.json()

        if res['numberReturned'] == 0:
            retry = 1
            while retry <= retries:
                retry += 1
                lat = row.latitude_wgs84
                lon = row.longitude_wgs84
                
                bbox = f'{lon-(buffer + retry*buffer_increment)},{lat-(buffer + retry*buffer_increment)},{lon+(buffer + retry*buffer_increment)},{lat+ (buffer + retry*buffer_increment)}'
                url = f'{base_url}/{service}/items?f={output_format}&bbox={bbox}' 
                res = call_geoconnex(url)

                if res['numberReturned'] > 0:
                    break
                
                    
        # if more than one object is returned, notify the user and then skip the site.
        if res['numberReturned'] != 1:    
            print(f'Gauge {row.STREAM_ID} returned {res["numberReturned"]} features. Skipping because I dont know what to do with this')
            errors[row.STREAM_ID] = res
            continue
    
        
        attributes[idx] = res['features'][0]['properties'][result_key] 

    return attributes, errors
        

In [15]:
attribs, errors = get_geoconnex_metadata(df, 'hu06', 'huc6')

  0%|          | 0/539 [00:00<?, ?it/s]

In [16]:
print(f'Found {len(errors)} errors')

Found 0 errors


In [18]:
df['huc06'] = df.index.map(attribs)

In [19]:
attribs, errors = get_geoconnex_metadata(df, 'hu12', 'huc12')

  0%|          | 0/539 [00:00<?, ?it/s]

In [20]:
df['huc12'] = df.index.map(attribs)

In [88]:

attribs, errors = get_geoconnex_metadata(df, 'mainstems', 'head_nhdpv2_comid', buffer=0.0001, retries=10)


  0%|          | 0/539 [00:00<?, ?it/s]

Gauge STREAM-gauge-1637 returned 0 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1639 returned 0 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1671 returned 0 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1693 returned 2 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1698 returned 0 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1704 returned 0 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1705 returned 0 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1708 returned 0 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1711 returned 2 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1717 returned 0 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1718 returned 0 features. Skipp

In [89]:
cleaned_attribs = {k: v.split('/')[-1] for k, v in attribs.items()}
print(f'Found {len(cleaned_attribs)} NHD matches')

Found 474 NHD matches


In [90]:
df['nhdv2_comid'] = df.index.map(cleaned_attribs)

In [91]:
print(f'{len(df[df.nhdv2_comid.isna()])} are still unknown')

65 are still unknown


Many gauges were not intersecting with NHD features. Increase the buffer for these location and try again.

In [72]:
attribs, errors = get_geoconnex_metadata(df[df.nhdv2_comid.isna()],
                                        'mainstems',
                                         'head_nhdpv2_comid',
                                         buffer=0.01)



  0%|          | 0/139 [00:00<?, ?it/s]

Gauge STREAM-gauge-1637 returned 0 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1645 returned 2 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1646 returned 3 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1666 returned 2 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1670 returned 3 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1692 returned 2 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1693 returned 2 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1704 returned 0 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1708 returned 0 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1711 returned 2 features. Skipping because I dont know what to do with this
Gauge STREAM-gauge-1712 returned 2 features. Skipp

In [73]:
cleaned_attribs = {k: v.split('/')[-1] for k, v in attribs.items()}
print(f'Found {len(cleaned_attribs)} NHD matches')

Found 69 NHD matches


In [77]:
# Only update rows where new_values is not NaN
new_values= df.index.map(cleaned_attribs)
mask = ~pandas.isna(new_values)
df.loc[mask, 'nhdv2_comid'] = new_values[mask]

In [78]:
print(f'{len(df[df.nhdv2_comid.isna()])} are still unknown')

70 are still unknown


In [94]:
df = df.drop(columns='geometry')
df

,STREAM_ID,SourceID,site name,source,latitude_wgs84,longitude_wgs84,State Code,State,drainagearea_sqkm,huc06,huc12,nhdv2_comid
0,STREAM-gauge-1616,01427195,"Equinunk Creek near Dillontown, PA",USGS,41.840278,-75.238333,PA,Pennsylvania,121.470531,020401,020401010402,2617668
1,STREAM-gauge-1617,01427207,DELAWARE RIVER AT LORDVILLE NY,USGS,41.867278,-75.213750,PA,Pennsylvania,4118.084100,020401,020401010406,2612780
2,STREAM-gauge-1618,01427510,DELAWARE RIVER AT CALLICOON NY,USGS,41.756750,-75.057417,PA,Pennsylvania,4713.781800,020401,020401010506,2612780
3,STREAM-gauge-1619,01428750,"West Branch Lackawaxen River near Aldenville, PA",USGS,41.674530,-75.376013,PA,Pennsylvania,105.153594,020401,020401030103,2739724
4,STREAM-gauge-1620,01429000,"West Branch Lackawaxen River at Prompton, PA",USGS,41.587218,-75.326829,PA,Pennsylvania,154.622403,020401,020401030103,2739724
...,...,...,...,...,...,...,...,...,...,...,...,...
534,STREAM-gauge-2175,99410012-001,Pigeon Creek DWS Bentleyville Rd,Pennsylvania Department of Environmental Prote...,40.138665,-79.983799,PA,Pennsylvania,106.710000,050200,050200050804,3786421
535,STREAM-gauge-2176,99416516-001,Smith Creek UPS Cook Ave,Pennsylvania Department of Environmental Prote...,39.889337,-80.185112,PA,Pennsylvania,27.700000,050200,050200050305,3788563
536,STREAM-gauge-2177,99681726-001,Raccoon Creek at Golf Course Road,Pennsylvania Department of Environmental Prote...,40.608930,-80.301372,PA,Pennsylvania,445.000000,050301,050301010207,3823425
537,STREAM-gauge-2178,99685860-001,Traverse Creek at SR18,Pennsylvania Department of Environmental Prote...,40.503509,-80.423835,PA,Pennsylvania,37.800000,050301,050301010205,3820555


In [95]:
df.to_parquet('data_parquet/metadata.parquet')